<div style = "text-align: left"><font size = 8 color = "#0000FF"#0000FFface = "verdana"><b>Tesco customer churn project</b></font></div>
<div style = "text-align: leftt"><font><i>By Jerry Aremu</i></font></div>
<div style = "text-align: leftt"><font>28th February 2023</font></div>

### import requred libraries

In [11]:
import pandas as pd
import os

### notebook custom format

In [12]:
import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.max_colwidth = None
pd.set_option("display.float_format", lambda x: '%.2f' % x)

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))



### Data Ingestion

In [13]:
data_folder = "data"
file_name = "telco_customer_churn.csv"

In [14]:
main_path = os.getcwd()

In [15]:
file_path = os.path.join(os.path.join(main_path, data_folder), file_name)
file_path

'C:\\Users\\jerea\\OneDrive\\Documents\\datahackerman_workspace\\telco_customer_churn\\data\\telco_customer_churn.csv'

In [16]:
telco_data = pd.read_csv(file_path)

In [17]:
telco_data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


### Data Inspection and Exploratory Analysis

#### Data inspection

In [18]:
# check the statistical summary of the data
telco_data.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.00,7043.00,7043.00
mean,0.16,32.37,64.76
std,0.37,24.56,30.09
min,0.00,0.00,18.25
25%,0.00,9.00,35.50
50%,0.00,29.00,70.35
75%,0.00,55.00,89.85
max,1.00,72.00,118.75


In [19]:
# check for data type mismatch and missing data
telco_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [20]:
# check the size of the data
telco_data.shape

(7043, 21)

#### Data exploration

##### Data Inspection findings:
1. Data appears to have been pre processed 
2. No missing data observed, and the 
3. object types for the data field appears correct


### Model developement

In [21]:
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularDataset, TabularPredictor

In [24]:
telco_data.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [25]:
# select all columns except the one that is to be predicted
X = telco_data[['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges']]

In [29]:
#drop the column id field
telco_data_wtout_id = telco_data.drop(["customerID"], axis=1)

In [30]:
telco_data_wtout_id.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [32]:
y = telco_data_wtout_id["Churn"]

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33, random_state=42)

In [35]:
X_train.shape, X_test.shape

((4718, 20), (2325, 20))

### Train the model

In [37]:
training_data, testing_data = train_test_split(telco_data_wtout_id, test_size = 0.33, random_state = 42)

In [38]:
training_data.shape, testing_data.shape

((4718, 20), (2325, 20))

In [40]:
label = "Churn"
print("Summary of class variable: \n", training_data[label].describe())


Summary of class variable: 
 count     4718
unique       2
top         No
freq      3477
Name: Churn, dtype: object


In [41]:
save_path = 'models'
predictor = TabularPredictor(label=label, path =save_path).fit(training_data)

Beginning AutoGluon training ...
AutoGluon will save models to "models\"
AutoGluon Version:  0.7.0
Python Version:     3.9.16
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Train Data Rows:    4718
Train Data Columns: 19
Label Column: Churn
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  ['No', 'Yes']
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = Yes, class 0 = No
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive (Yes) vs negative (No) class.
	To explicitly set the positive_class, either rename classes to 1 and 0, or specify positive_class in Predictor init.
Using Feature Generators to preproce

In [42]:
predictor.leaderboard(silent=True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT,0.81,0.01,0.85,0.01,0.85,1,True,3
1,WeightedEnsemble_L2,0.81,0.01,1.34,0.00,0.49,2,True,14
2,CatBoost,0.80,0.01,13.20,0.01,13.20,1,True,7
3,LightGBM,0.80,0.01,0.63,0.01,0.63,1,True,4
4,NeuralNetTorch,0.80,0.01,5.82,0.01,5.82,1,True,12
5,XGBoost,0.79,0.01,0.88,0.01,0.88,1,True,11
6,NeuralNetFastAI,0.79,0.03,7.47,0.03,7.47,1,True,10
7,LightGBMLarge,0.79,0.01,1.13,0.01,1.13,1,True,13
8,RandomForestGini,0.78,0.09,0.92,0.09,0.92,1,True,5
9,ExtraTreesGini,0.78,0.11,0.73,0.11,0.73,1,True,8


NameError: name 'train_data' is not defined